# Problem Statement 

The problem statement is to detect intrusions from a tcp dump.The data is provided in csv files so no feature extraction is involved 

In [1]:
from data import fetch_data 
from data import feature_engineering
from models.model import Model
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import os
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.externals import joblib

from Para_learn import paralearn
import pickle
from flask import request
from cross_val import k_fold_crossval
%matplotlib inline 
root=os.getcwd()

/home/anurag/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
x_train,y_train,x_test,y_test = fetch_data(root,remove_duplicates=True,binary=True)

FileNotFoundError: File b'/home/anurag/Desktop/DS_Training/data/session_1_data_train.csv' does not exist

In [ ]:
x_train = feature_engineering(x_train,do_normalization=False)
x_test = feature_engineering(x_test,do_normalization=False)
print('Shape of training data after feature engineering is {}'.format(x_train.shape))
print ('Shape of test data after feature engineering is {}'.format(x_test.shape))

In [ ]:
y_train.value_counts()

# Sampling 

# Modelling in progress

In [ ]:

clf=Model(x_train,y_train)


In [ ]:
#NAIVE BAYES
nb=clf.mnbayes(x_train,y_train)
predict_train_nb=nb.predict(x_train)
conf_matrix_nb=confusion_matrix(list(y_train),(predict_train_nb).round())
f1_score_train_nb=f1_score(list(y_train),(predict_train_nb).round())
print (conf_matrix_nb)
print('The f1 score for NAIVE BAYES model on training data is {}'.format(f1_score_train_nb))


In [ ]:
cross_val_score_nb=k_fold_crossval(nb,x_train,y_train,6)

In [ ]:
#SVM 
svm=clf.svm(x_train,y_train)
predict_train_svm=svm.predict(x_train)
conf_matrix_svm=confusion_matrix(list(y_train),(predict_train_svm).round())
f1_score_train_svm=f1_score(list(y_train),(predict_train_svm).round())
print (conf_matrix_svm)
print('The f1 score for SUPPORT VECTOR MACHINES model on training data is {}'.format(f1_score_train_svm))


In [ ]:
cross_val_score_svm=k_fold_crossval(svm,x_train,y_train,6)

In [ ]:
#XGBOOST 
xgb,x_dtrain,x_dtest=clf.xgboost(x_train,y_train,x_test)
predict_train_xgb=xgb.predict(x_dtrain)
conf_matrix_xgb=confusion_matrix(list(y_train),(predict_train_xgb).round())
f1_score_train_xgb=f1_score(list(y_train),(predict_train_xgb).round())
print (conf_matrix_xgb)
print('The f1 score for XGBOOST model on training data is {}'.format(f1_score_train_xgb))


In [ ]:
#Checking whether this is a overfit on the training data 


In [ ]:
#RANDOMFOREST 
rf=clf.randforest(x_train,y_train)
predict_train_rf=rf.predict(x_train)
conf_matrix_rf=confusion_matrix(list(y_train),(predict_train_rf).round())
f1_score_train_rf=f1_score(list(y_train),(predict_train_rf).round())
print (conf_matrix_rf)
print('The f1 score for RANDOM FOREST model on training data is {:.4f}'.format(f1_score_train_rf))


In [ ]:
# Cross validation to check whether the model is overfitting or not 
cross_val_score_rf=k_fold_crossval(rf,x_train,y_train,6)


# Model Selection 


After doing hyperparameter optimization for different models on the training data and computing a metric evaluating the generalization of a particular model,we can select which model will be our best bet for generalizing over to the unseen data 

# Hitting the API 

In [ ]:
#Random forest is the one. Lets save this model somewhere and hit the api to see what happens 
joblib.dump(rf,'random_forest.pkl')

In [ ]:
# Hitting the api 
root

In [ ]:
import pickle

In [ ]:
pickle.load(open('rf_test.pkl','rb'))

In [ ]:
pickle.dump(rf,open('rf_test.pkl','wb'),protocol=2)